In [25]:
import pandas as pd
import numpy as np
import collections

In [26]:
# This dataframe creates vectorized season data for each team

all_teams_pd = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MTeams.csv')
all_teams_list = all_teams_pd['Team_Name'].tolist()

def season_vector(team_id, year):
    return df.query('WTeamID == {} and Season == {}'.format(team_id, year)).values.tolist()

def compare_teams(team_id_1, team_id_2, year):
    team_1 = season_vector(team_id_1, year)
    team_2 = season_vector(team_id_2, year)
    comparison = [a - b for a, b in zip(team_1, team_2)]
    return comparison

def all_teams_season(year):
    season_dictionary = collections.defaultdict(list)
    for team in all_teams_list:
        team_id = all_teams_pd[all_teams_pd['Team_Name'] == team].values[0][0]
        team_vector = season_vector(team_id, year)
        season_dictionary[team_id] = team_vector
    return season_dictionary

def codify_home_away(row_num):
    if (row_num == 'H'):
        home = 1
    if (row_num == 'A'):
        home = -1
    if (row_num == 'N'):
        home = 0
    return home

TypeError: unsupported operand type(s) for -: 'list' and 'list'